# Shark Habitat & Movement Model — v2 (expanded drivers + apps)

This version adds **eddy polarity**, **EKE**, **SST fronts**, **distance to eddy core/edge**, **bathymetry + distance to shelf break**, and **mixed layer depth (MLD)** to the habitat model. It also ships a **Streamlit** and a **Bokeh** dashboard (see project folder) for interactive exploration.

**Tip:** All fields are easily swappable with real **SWOT (SSH)**, **MODIS/PACE (Chl)**, and **SST** arrays.

In [ ]:

# Imports & setup
import math, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

# Load the environment bundle prepared for this notebook (or replace with your own)
env = np.load("/mnt/data/env_fields_v2.npz")
lons = env["lons"]; lats = env["lats"]
SSH = env["SSH"]; U = env["U"]; V = env["V"]; SPEED = env["SPEED"]
SST = env["SST"]; CHL = env["CHL"]
W = env["W"]; EDDY_CORE_MASK = env["EDDY_CORE_MASK"].astype(bool); EDDY_EDGE_MASK = env["EDDY_EDGE_MASK"].astype(bool)
WARM_MASK = env["WARM_MASK"].astype(bool)
EKE = env["EKE"]; SST_FRONT = env["SST_FRONT"]
BATHY = env["BATHY"]; SHELF_MASK = env["SHELF_MASK"].astype(bool)
MLD = env["MLD"]
DIST_TO_CORE_M = env["DIST_TO_CORE_M"]; DIST_TO_EDGE_M = env["DIST_TO_EDGE_M"]; DIST_TO_SHELF_M = env["DIST_TO_SHELF_M"]

LAT_MIN = float(env["LAT_MIN"]); LAT_MAX = float(env["LAT_MAX"])
LON_MIN = float(env["LON_MIN"]); LON_MAX = float(env["LON_MAX"])
dx_m = float(env["dx_m"]); dy_m = float(env["dy_m"]); f = float(env["f"]); g = float(env["g"])

NY, NX = SSH.shape
LON, LAT = np.meshgrid(lons, lats)

def bilinear(field2d, lon, lat):
    # Safe bilinear interpolation on our regular grid
    lon = np.clip(lon, lons[0], lons[-1]); lat = np.clip(lat, lats[0], lats[-1])
    ix = np.searchsorted(lons, lon) - 1; iy = np.searchsorted(lats, lat) - 1
    ix = np.clip(ix, 0, NX-2); iy = np.clip(iy, 0, NY-2)
    x0, x1 = lons[ix], lons[ix+1]; y0, y1 = lats[iy], lats[iy+1]
    wx = (lon - x0) / (x1 - x0 + 1e-12); wy = (lat - y0) / (y1 - y0 + 1e-12)
    f00 = field2d[iy, ix]; f10 = field2d[iy, ix+1]; f01 = field2d[iy+1, ix]; f11 = field2d[iy+1, ix+1]
    return (1-wx)*(1-wy)*f00 + wx*(1-wy)*f10 + (1-wx)*wy*f01 + wx*wy*f11


### Visual checks

In [ ]:

plt.figure(figsize=(7,5))
plt.title("SSH anomaly [m] & eddy cores (x)")
plt.imshow(SSH, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
yy, xx = np.where(EDDY_CORE_MASK); sel = (np.arange(yy.size) % 30) == 0
plt.scatter(lons[xx[sel]], lats[yy[sel]], s=10, marker='x')
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()

plt.figure(figsize=(7,5))
plt.title("SST fronts |∇SST|")
plt.imshow(SST_FRONT, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()

plt.figure(figsize=(7,5))
plt.title("EKE [m$^2$ s$^{-2}$]")
plt.imshow(EKE, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()

plt.figure(figsize=(7,5))
plt.title("Bathymetry [m] & 200 m shelf break (dots)")
plt.imshow(BATHY, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
yy, xx = np.where(SHELF_MASK); sel = (np.arange(yy.size) % 50) == 0
plt.scatter(lons[xx[sel]], lats[yy[sel]], s=8)
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()


### Tag ingest / simulation (replace with your live tag feed)

In [ ]:

# Here we re-use the synthetic tag if present, otherwise generate one.
tag_csv = Path('/mnt/data/example_tag_data.csv')
if tag_csv.exists():
    tag_df = pd.read_csv(tag_csv, parse_dates=['time'])
else:
    # simple seed track if the CSV isn't there
    rng = np.random.default_rng(1)
    n_steps, dt_hours = 120, 6.0
    dt = dt_hours*3600.0
    lon, lat = -71.0, 35.5
    track = []
    for k in range(n_steps):
        u = bilinear(U, lon, lat); v = bilinear(V, lon, lat)
        lon += (u*dt)/dx_m + 0.02*rng.normal()
        lat += (v*dt)/dy_m + 0.02*rng.normal()
        lon = float(np.clip(lon, LON_MIN, LON_MAX)); lat = float(np.clip(lat, LAT_MIN, LAT_MAX))
        depth = 200.0
        t = pd.Timestamp('2025-05-01') + pd.to_timedelta(k*dt_hours, unit='h')
        track.append([t, lon, lat, depth, u, v])
    tag_df = pd.DataFrame(track, columns=['time','lon','lat','depth_m','u_ms','v_ms'])
    tag_df.to_csv(tag_csv, index=False)

print(tag_df.head())


### Expanded features & HSI

In [ ]:

# Build feature extraction, now including the new drivers
def extract_features(lon_pts, lat_pts):
    vals = []
    for lo, la in zip(lon_pts, lat_pts):
        sst = bilinear(SST, lo, la)
        chl = bilinear(CHL, lo, la)
        ssh = bilinear(SSH, lo, la)
        eke = bilinear(EKE, lo, la)
        fr = bilinear(SST_FRONT, lo, la)
        bathy = bilinear(BATHY, lo, la)
        mld = bilinear(MLD, lo, la)
        # distances (meters)
        d_core = bilinear(DIST_TO_CORE_M, lo, la)
        d_edge = bilinear(DIST_TO_EDGE_M, lo, la)
        d_shelf = bilinear(DIST_TO_SHELF_M, lo, la)
        # preferences / proximities
        sst_pref = math.exp(-((sst - 22.0)/2.5)**2)
        chl_pref = math.exp(-((chl - 0.45)/0.25)**2)
        mld_pref = math.exp(-((mld - 40.0)/15.0)**2)
        # proximity kernels (meters -> unitless 0..1)
        def prox(d, L=80000.0):  # 80 km e-folding
            return math.exp(-max(d, 0.0)/L)
        core_prox = prox(d_core, 60000.0)
        edge_prox = prox(d_edge, 50000.0)
        shelf_prox = prox(d_shelf, 30000.0)
        vals.append([sst, chl, ssh, eke, fr, bathy, mld, d_core, d_edge, d_shelf,
                     sst_pref, chl_pref, mld_pref, core_prox, edge_prox, shelf_prox])
    arr = np.array(vals, dtype=float)
    # standardize continuous physical magnitudes (first 10 columns)
    mu = arr[:, :10].mean(axis=0); sig = arr[:, :10].std(axis=0) + 1e-9
    arr[:, :10] = (arr[:, :10] - mu) / sig
    return arr, mu, sig

# Case-control (used vs available endpoints)
used_lon = tag_df['lon'].values[1:]
used_lat = tag_df['lat'].values[1:]
n = used_lon.size
K = 5  # available per used
rng = np.random.default_rng(2)
avail_lon, avail_lat = [], []
for i in range(n):
    lo0, la0 = tag_df['lon'].values[i], tag_df['lat'].values[i]
    props_lo = lo0 + 0.6*np.random.normal(size=K)
    props_la = la0 + 0.6*np.random.normal(size=K)
    props_lo = np.clip(props_lo, LON_MIN, LON_MAX)
    props_la = np.clip(props_la, LAT_MIN, LAT_MAX)
    avail_lon.append(props_lo); avail_lat.append(props_la)
avail_lon = np.array(avail_lon).reshape(-1); avail_lat = np.array(avail_lat).reshape(-1)

X_used, mu, sig = extract_features(used_lon, used_lat)
X_av, _, _ = extract_features(avail_lon, avail_lat)

X = np.vstack([X_used, X_av])
y = np.hstack([np.ones(X_used.shape[0]), np.zeros(X_av.shape[0])])

# Add intercept
X = np.hstack([np.ones((X.shape[0], 1)), X])

def fit_logit_newton(X, y, lam=1e-2, max_iter=80):
    w = np.zeros(X.shape[1])
    for _ in range(max_iter):
        z = X @ w
        p = 1 / (1 + np.exp(-z))
        W = p * (1 - p)
        grad = X.T @ (y - p) - lam * w
        H = -(X.T * W) @ X - lam * np.eye(X.shape[1])
        try:
            step = np.linalg.solve(H, grad)
        except np.linalg.LinAlgError:
            step = np.linalg.pinv(H) @ grad
        w_new = w - step
        if np.linalg.norm(w_new - w) < 1e-6:
            w = w_new; break
        w = w_new
    return w

coef = fit_logit_newton(X, y, lam=2e-2, max_iter=120)
print("Coefficients length:", coef.shape[0])

def hsi_prob(lon, lat, coef, mu, sig):
    # recompute feature vector at a single point, then logistic probability
    sst = bilinear(SST, lon, lat); chl = bilinear(CHL, lon, lat); ssh = bilinear(SSH, lon, lat)
    eke = bilinear(EKE, lon, lat); fr = bilinear(SST_FRONT, lon, lat)
    bathy = bilinear(BATHY, lon, lat); mld = bilinear(MLD, lon, lat)
    d_core = bilinear(DIST_TO_CORE_M, lon, lat); d_edge = bilinear(DIST_TO_EDGE_M, lon, lat); d_shelf = bilinear(DIST_TO_SHELF_M, lon, lat)
    sst_pref = math.exp(-((sst - 22.0)/2.5)**2)
    chl_pref = math.exp(-((chl - 0.45)/0.25)**2)
    mld_pref = math.exp(-((mld - 40.0)/15.0)**2)
    def prox(d, L=80000.0): return math.exp(-max(d, 0.0)/L)
    core_prox = prox(d_core, 60000.0); edge_prox = prox(d_edge, 50000.0); shelf_prox = prox(d_shelf, 30000.0)
    arr = np.array([sst, chl, ssh, eke, fr, bathy, mld, d_core, d_edge, d_shelf,
                    sst_pref, chl_pref, mld_pref, core_prox, edge_prox, shelf_prox], dtype=float)
    # standardize first 10 physical magnitudes
    arr[:10] = (arr[:10] - mu) / sig
    x = np.hstack([1.0, arr])  # intercept
    z = float(x @ coef)
    return 1.0 / (1.0 + math.exp(-z))

# Build a gridded HSI
HSI = np.zeros_like(SSH)
for j in range(SSH.shape[0]):
    for i in range(SSH.shape[1]):
        HSI[j, i] = hsi_prob(lons[i], lats[j], coef, mu, sig)

plt.figure(figsize=(7,5))
plt.title("Fitted Habitat Suitability (with expanded drivers)")
plt.imshow(HSI, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()


### Forecast with expanded HSI

In [ ]:

# Forecast parameters
dt_hours = 6.0; dt = dt_hours*3600.0
N_fore = 24; N_ens = 40

last_lon = float(tag_df['lon'].values[-1]); last_lat = float(tag_df['lat'].values[-1])

def predict_traj(lon0, lat0, coef, mu, sig, seed):
    rng = np.random.default_rng(seed)
    traj = [(lon0, lat0)]
    vx_prev, vy_prev = 0.0, 0.0
    for _ in range(N_fore):
        u = bilinear(U, lon0, lat0); v = bilinear(V, lon0, lat0)
        eps = 0.02
        # finite-diff gradient of HSI probability
        h_e = hsi_prob(min(lon0+eps, LON_MAX), lat0, coef, mu, sig)
        h_w = hsi_prob(max(lon0-eps, LON_MIN), lat0, coef, mu, sig)
        h_n = hsi_prob(lon0, min(lat0+eps, LAT_MAX), coef, mu, sig)
        h_s = hsi_prob(lon0, max(lat0-eps, LAT_MIN), coef, mu, sig)
        dhdx = (h_e - h_w) / (2*eps*dx_m); dhdy = (h_n - h_s) / (2*eps*dy_m)
        bias_u = +500.0 * dhdx; bias_v = +500.0 * dhdy
        vx_m = 0.6*(u + bias_u) + 0.3*vx_prev + 0.05*rng.normal()
        vy_m = 0.6*(v + bias_v) + 0.3*vy_prev + 0.05*rng.normal()
        lon0 += (vx_m * dt) / dx_m; lat0 += (vy_m * dt) / dy_m
        lon0 = float(np.clip(lon0, LON_MIN, LON_MAX)); lat0 = float(np.clip(lat0, LAT_MIN, LAT_MAX))
        vx_prev, vy_prev = vx_m, vy_m
        traj.append((lon0, lat0))
    return np.array(traj)

trajectories = [predict_traj(last_lon, last_lat, coef, mu, sig, i) for i in range(N_ens)]

# Plot
plt.figure(figsize=(7,5))
plt.title("Observed (tail) and Forecast Ensemble over HSI")
plt.imshow(HSI, origin='lower', extent=[LON_MIN, LON_MAX, LAT_MIN, LAT_MAX])
tail = tag_df.tail(16)
plt.plot(tail['lon'].values, tail['lat'].values)
for tr in trajectories:
    plt.plot(tr[:,0], tr[:,1], linewidth=0.8, linestyle='--')
plt.xlabel("Longitude"); plt.ylabel("Latitude"); plt.tight_layout(); plt.show()
